# Plot rt_is.file in 3D

In [6]:
import numpy as np
from pathlib import Path

# Provide the folder path where the file is located
pyptv_folder = Path.cwd().parent / 'pyptv'
folder_path = pyptv_folder / 'res'  # Replace with the actual folder path
cal_path = pyptv_folder / 'cal'
ori_files = cal_path.rglob('*.ori')

# Construct the full file path
file_path = f'{folder_path}/rt_is.123456789'

# Read the file using numpy
data = np.loadtxt(file_path, skiprows=1)



# print(data)

In [8]:
data[1:]

array([[ 2.000000e+00,  3.122594e+03,  1.063382e+03,  1.829098e+03,
         5.000000e+00,  5.000000e+00,  5.000000e+00,  5.000000e+00],
       [ 3.000000e+00,  1.148487e+03,  2.407450e+02,  2.641245e+03,
         8.000000e+00,  6.000000e+00,  9.000000e+00,  7.000000e+00],
       [ 4.000000e+00,  1.743363e+03,  2.320908e+03,  7.900560e+02,
         4.000000e+00,  4.000000e+00,  3.000000e+00,  4.000000e+00],
       [ 5.000000e+00, -7.575240e+02,  3.196980e+03,  8.530860e+02,
         0.000000e+00,  2.000000e+00,  1.000000e+00,  0.000000e+00],
       [ 6.000000e+00,  3.056435e+03,  3.473787e+03,  1.802472e+03,
         2.000000e+00,  1.000000e+00,  2.000000e+00,  1.000000e+00],
       [ 7.000000e+00,  4.062387e+03,  2.436780e+02, -7.766200e+01,
         9.000000e+00,  7.000000e+00,  7.000000e+00,  9.000000e+00],
       [ 8.000000e+00,  9.610550e+02,  8.562920e+02, -4.033990e+02,
         7.000000e+00,  8.000000e+00,  6.000000e+00,  8.000000e+00],
       [ 9.000000e+00, -1.384288e+03,  6.

In [9]:
import plotly.graph_objs as go
import plotly.express as px

# Filter the rows where the last column is not -1
filtered_data = data[np.sum(data[:, -4:] == -1, axis=1) < 3]

# Extract the columns for the 3D scatter plot from the filtered data
x = filtered_data[:, 1]
y = filtered_data[:, 2]
z = filtered_data[:, 3]

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='markers')])

# Set plot title and labels
fig.update_layout(title='3D Scatter Plot', scene=dict(
    xaxis_title='Z',
    yaxis_title='X',
    zaxis_title='Y'
))


for f in ori_files:
    with open(f, 'r') as file:
        # Read the first line for camera position
        cam_pos = np.array(file.readline().strip().split(), dtype=float)
        # Read the second line for camera angles
        cam_angles = np.array(file.readline().strip().split(), dtype=float)
    
    # Calculate the direction vector from the angles
    direction = np.array([
        np.cos(cam_angles[1]) * np.cos(cam_angles[0]),
        np.sin(cam_angles[1]),
        np.cos(cam_angles[1]) * np.sin(cam_angles[0])
    ])
    
    # Plot the camera position
    fig.add_trace(go.Scatter3d(
        x=[cam_pos[0]], y=[cam_pos[1]], z=[cam_pos[2]],
        mode='markers',
        marker=dict(size=5, color='red'),
        name='Camera Position'
    ))
    
    # Plot the direction arrow
    fig.add_trace(go.Scatter3d(
        x=[cam_pos[0], cam_pos[0] + direction[0]],
        y=[cam_pos[1], cam_pos[1] + direction[1]],
        z=[cam_pos[2], cam_pos[2] + direction[2]],
        mode='lines',
        line=dict(color='red', width=5),
        name='Camera Direction'
    ))

# Show the plot
fig.show()

In [3]:
z

array([2701.149, 1063.382,  240.745, 2320.908, 3196.98 , 3473.787,
        243.678,  856.292,  689.355, 3208.971])